# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [17]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
import kaleido
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, regexp_replace, transform, count, to_date, col

spark = SparkSession.builder.appName("LightcastData").getOrCreate()

df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .option("multiLine", "true") \
               .option("escape", "\"") \
               .csv("../lightcast_job_postings.csv")

df.printSchema()

df.show(5, truncate=False)


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: date (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: date (nullable = true)
 |-- EXPIRED: date (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: date (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (nullable

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [18]:
filtered_df = df.filter((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM") > 0))

pdf = filtered_df.select("EMPLOYMENT_TYPE_NAME", "SALARY_FROM").toPandas()

soft_colors = ['#87CEEB', '#ADD8E6', '#B0E0E6', '#AFEEEE', '#E0FFFF', '#D8BFD8']

fig = px.box(
    pdf,
    x="EMPLOYMENT_TYPE_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Employment Type",
    color="EMPLOYMENT_TYPE_NAME",
    points="all",
    color_discrete_sequence=soft_colors
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    xaxis=dict(
        title="Employment Type",
        title_font=dict(size=16, family="Times New Roman", color="black"),
        tickfont=dict(size=12, color="gray"),
        showgrid=False
    ),
    yaxis=dict(
        title="Starting Salary (From)",
        title_font=dict(size=16, family="Times New Roman", color="black"),
        tickfont=dict(size=12, color="gray"),
        showgrid=True,
        gridcolor='lightgray'
    ),
    plot_bgcolor="white",
    legend=dict(
        font=dict(family="Times New Roman", size=12),
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig.show()

![Salary Distribution](output/1.png)

The box plot shows clear differences in starting salaries by employment type. Full-time employment (≥ 32 hours) possesses higher median wages and greater range, suggesting greater earning ability and variability. Part-time employment, particularly with fewer than or equal to 32 hours, suggests lower and more restricted salary distributions, suggesting more stable lower pay. 

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [19]:
# Your code for 2nd question here
filtered_df = df.filter(
    (col("SALARY_FROM").isNotNull()) &
    (col("SALARY_FROM") > 0) &
    (col("NAICS2_NAME").isNotNull())
)

pdf = filtered_df.select("NAICS2_NAME", "SALARY_FROM").toPandas()

industry_order = pdf.groupby("NAICS2_NAME")["SALARY_FROM"].median().sort_values(ascending=False).index
pdf["NAICS2_NAME"] = pd.Categorical(pdf["NAICS2_NAME"], categories=industry_order, ordered=True)

soft_colors = ['#6EC6FF', '#90CAF9', '#64B5F6', '#4FC3F7', '#81D4FA', '#29B6F6']

fig = px.box(
    pdf,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Industry",
    points="all",
    color_discrete_sequence=soft_colors
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    xaxis=dict(
        title="Industry",
        title_font=dict(size=16, color="black"),
        tickangle=-45,  
        tickfont=dict(size=11, color="black"),
        tickmode='linear'
    ),
    yaxis=dict(
        title="Starting Salary (From)",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=12, color="black"),
        showgrid=True,
        gridcolor='lightgray'
    ),
    plot_bgcolor="white"
)

fig.show()


![Salary Distribution](output/2.png)

The box plot reveals tremendous variation in starting salaries across industries. Professional, Scientific, and Technical Services and Finance and Insurance are industries that, on average, have higher median starting salaries because of the high level of skills demanded in these occupations and paid for in these occupations. Accommodation and Food Services and Arts, Entertainment, and Recreation are industries that have lower and more spread-out salary distributions, which represent more balanced and lower aggregate pay.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [20]:
df_posted = df.withColumn("POSTED_DATE", to_date(col("POSTED")))

daily_counts = df_posted.groupBy("POSTED_DATE").agg(count("*").alias("NUM_POSTINGS"))

pdf = daily_counts.orderBy("POSTED_DATE").toPandas()

fig = px.line(
    pdf,
    x="POSTED_DATE",
    y="NUM_POSTINGS",
    title="Job Posting Trends Over Time",
    markers=True,
    line_shape="linear"
)

fig.update_traces(line_color='#1E90FF')

fig.update_layout(
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    xaxis=dict(
        title="Posted Date",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=12, color="gray"),
        showgrid=False
    ),
    yaxis=dict(
        title="Number of Job Postings",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=12, color="gray"),
        showgrid=True,
        gridcolor='lightgray'
    ),
    plot_bgcolor="white"
)

fig.show()

![Salary Distribution](output/3.png)

The line graph illustrates the daily fluctuation of job postings from May to October 2024. Overall, the trend reflects periodic highs and lows, which can be suggestive of cyclical recruitment patterns, i.e., recruitment campaigns, weekends, or holiday periods. Dominant peaks in postings can virtually undoubtedly be credited to planned hiring campaigns, while troughs can indicate times of laggings in the job market.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [21]:
title_counts = (
    df.filter(col("TITLE_NAME").isNotNull())
      .groupBy("TITLE_NAME")
      .agg(count("*").alias("JOB_COUNT"))
)

top10_titles = title_counts.orderBy(col("JOB_COUNT").desc()).limit(10).toPandas()

top10_titles = top10_titles.sort_values(by="JOB_COUNT", ascending=True)

custom_colors = ['#81D4FA', '#4FC3F7', '#29B6F6', '#03A9F4', '#039BE5',
                 '#0288D1', '#0277BD', '#01579B', '#00B8D4', '#00ACC1']

fig = px.bar(
    top10_titles,
    x="JOB_COUNT",
    y="TITLE_NAME",
    orientation="h",
    title="Top 10 Job Titles by Count",
    color="TITLE_NAME",
    color_discrete_sequence=custom_colors,
    text="JOB_COUNT"
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    xaxis=dict(
        title="Job Count",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=12, color="gray"),
        showgrid=True,
        gridcolor="lightgray"
    ),
    yaxis=dict(
        title="Job Title",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=12, color="black"),
        showgrid=False
    ),
    plot_bgcolor="white",
    showlegend=False
)

fig.update_traces(textposition='outside')

fig.show()

![Salary Distribution](output/4.png)

The bar graph shows that "Data Analysts" is the most job title posted by far, at over 8,500 postings, an indication of the high and steady demand for data professionals. Other top-ranking roles are "Business Intelligence Analysts" and "Enterprise Architects," which signifies that firms are looking at data infrastructure, analysis, and planning strategically in the process of recruiting.

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [22]:
# Your code for 5th question here
remote_counts = (
    df.filter(col("REMOTE_TYPE_NAME").isNotNull())
      .groupBy("REMOTE_TYPE_NAME")
      .agg(count("*").alias("JOB_COUNT"))
)

pdf = remote_counts.toPandas()

fig = px.pie(
    pdf,
    names="REMOTE_TYPE_NAME",
    values="JOB_COUNT",
    title="Remote vs On-Site Job Postings",
    color_discrete_sequence=['#4FC3F7', '#00BCD4', '#0288D1', '#FF7043'],
    hole=0.4  
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    plot_bgcolor="white",
    legend_title_text="Job Type"
)

fig.show()

![Salary Distribution](output/5.png)

The pie chart shows that most job postings (over 78%) do not specify whether the job is remote or on-site, which may be due to variations in job posting standards. Of those listings that do specify, remote and hybrid jobs combined account for around 20%, indicating a rising proportion of flexible work arrangements in the market.

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [23]:
# Your code for 6th question here
from pyspark.sql.functions import col, explode, split
import pandas as pd
import plotly.express as px

skills_df = df.filter((col("NAICS2_NAME").isNotNull()) & (col("SKILLS").isNotNull()))

skills_exploded = skills_df.withColumn("SKILL", explode(split(col("SKILLS"), ",\s*")))

skill_counts = (
    skills_exploded.groupBy("NAICS2_NAME", "SKILL")
    .count()
    .withColumnRenamed("count", "SKILL_COUNT")
)

skill_pdf = skill_counts.toPandas()

top_skills = (
    skill_pdf.groupby("SKILL")["SKILL_COUNT"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index.tolist()
)

filtered_pdf = skill_pdf[skill_pdf["SKILL"].isin(top_skills)]

fig = px.bar(
    filtered_pdf,
    x="NAICS2_NAME",
    y="SKILL_COUNT",
    color="SKILL",
    title="Top 5 Skills in Demand by Industry",
    text="SKILL_COUNT"
)

fig.update_layout(
    barmode="stack",
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    xaxis=dict(
        title="Industry",
        title_font=dict(size=16, color="black"),
        tickangle=-45,
        tickfont=dict(size=11, color="black")
    ),
    yaxis=dict(
        title="Skill Count",
        title_font=dict(size=16, color="black"),
        tickfont=dict(size=12, color="black"),
        showgrid=True,
        gridcolor="lightgray"
    ),
    plot_bgcolor="white"
)

fig.show()

<>:8: SyntaxWarning:

invalid escape sequence '\s'

<>:8: SyntaxWarning:

invalid escape sequence '\s'

/tmp/ipykernel_9741/803094578.py:8: SyntaxWarning:

invalid escape sequence '\s'



![Salary Distribution](output/6.png)

The stacked bar chart highlights the distribution of skill demand across various industries. It shows that Professional, Scientific, and Technical Services and Administrative and Support Services are the top two industries with the highest volume of skill mentions, indicating a broad and intensive requirement for skilled labor in those sectors.


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [24]:
# Your code for 7th question here
from pyspark.sql.functions import col, count, expr, percentile_approx
import pandas as pd
import plotly.express as px

salary_df = df.filter(
    (col("ONET_NAME").isNotNull()) &
    (col("SALARY_FROM").isNotNull()) &
    (col("SALARY_FROM") > 0)
)

median_salary_df = (
    salary_df.groupBy("ONET_NAME")
    .agg(
        percentile_approx("SALARY_FROM", 0.5).alias("MEDIAN_SALARY"),
        count("*").alias("POSTING_COUNT")
    )
)

pdf = median_salary_df.toPandas()

fig = px.scatter(
    pdf,
    x="ONET_NAME",
    y="MEDIAN_SALARY",
    size="POSTING_COUNT",
    title="Median Salary by ONET Occupation Type",
    color="ONET_NAME",
    size_max=60
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14),
    title_font=dict(family="Times New Roman", size=20, color="darkblue"),
    xaxis=dict(
        title="ONET Occupation",
        title_font=dict(size=16),
        tickangle=-45,
        tickfont=dict(size=11, color="black")
    ),
    yaxis=dict(
        title="Median Salary",
        title_font=dict(size=16),
        tickfont=dict(size=12, color="black"),
        showgrid=True,
        gridcolor="lightgray"
    ),
    plot_bgcolor="white",
    showlegend=False
)

fig.show()

![Salary Distribution](output/7.png)

The bubble chart displays salary data for one ONET occupation type, "Business Intelligence Analysts," reporting minimal available data in the database. While the chart shows a mean salary of roughly $88,000 for this job, the absence of other occupation categories makes it difficult to compare salary trends within the overall job market.

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [25]:
# Your code for 8th question here
from pyspark.sql.functions import col, count
import pandas as pd
import plotly.graph_objects as go

sankey_df = (
    df.filter((col("SOC_2021_2_NAME").isNotNull()) & (col("SOC_2021_3_NAME").isNotNull()))
      .groupBy("SOC_2021_2_NAME", "SOC_2021_3_NAME")
      .agg(count("*").alias("TRANSITIONS"))
)

pdf = sankey_df.toPandas()

all_nodes = pd.unique(pdf[["SOC_2021_2_NAME", "SOC_2021_3_NAME"]].values.ravel())
node_map = {name: idx for idx, name in enumerate(all_nodes)}

pdf["source_idx"] = pdf["SOC_2021_2_NAME"].map(node_map)
pdf["target_idx"] = pdf["SOC_2021_3_NAME"].map(node_map)

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=20,
        thickness=20,
        line=dict(color="gray", width=0.5),
        label=list(all_nodes),
        color="lightblue"
    ),
    link=dict(
        source=pdf["source_idx"],
        target=pdf["target_idx"],
        value=pdf["TRANSITIONS"],
        color="rgba(100, 149, 237, 0.4)"  
    )
)])

fig.update_layout(
    title_text="Career Pathway Trends by SOC Classification",
    font=dict(family="Times New Roman", size=14),
    title_font=dict(size=20, color="darkblue")
)

fig.show()

![Salary Distribution](output/8.png)

Based on the Sankey diagram, there is a direct path from Computer and Mathematical Occupations to Mathematical Science Occupations with no flows in between. This would entail a highly specialized career trajectory within the dataset, where experts in overall computing roles might specialize further into mathematical science roles.